# MTH9821 Midterm 2023 Solutions
Please contact me @ nicolas.buchwalder.baruchmfe@gmail.com if you have any questions/remarks

In [1]:
!pip install numba --quiet


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [2]:
from numba import jit

import numpy as np
import pandas as pd

from scipy.stats import norm
from functools import partial

from scipy.optimize import root_scalar

import matplotlib.pyplot as plt

In [3]:
round = lambda x: np.round(x, 4)

In [4]:
np.random.seed(0)

## Analytical formulas

In [5]:
def blackscholes_analytical(S, K, T, sigma, r, q, opt_type='call'):
    """
    Pricing vanilla european options with closed form solution
    """
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if opt_type == 'call':
        return round(S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))
    else:
        return round(K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1))

In [6]:
def blackscholes_greeks(S, K, T, sigma, r, q, opt_type='call'):
    """
    Computing greeks for vanilla european options with closed form solution
    """
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    n_d1 = norm.pdf(d1)
    
    exp_qT = np.exp(-q * T)
    exp_rT = np.exp(-r * T)
    
    delta = exp_qT * N_d1 if opt_type == 'call' else -exp_qT * norm.cdf(-d1)
    gamma = exp_qT * n_d1 / (S * sigma * np.sqrt(T))
    vega = S * np.sqrt(T) * exp_qT * n_d1
    theta = -(S * exp_qT * n_d1 * sigma / (2 * np.sqrt(T))) + (r * K * exp_rT * N_d2 - q * S * exp_qT * N_d1) if opt_type == 'call' else -(S * exp_qT * n_d1 * sigma / (2 * np.sqrt(T))) - (r * K * exp_rT * norm.cdf(-d2) - q * S * exp_qT * norm.cdf(-d1))
    rho = K * T * exp_rT * N_d2 if opt_type == 'call' else -K * T * exp_rT * norm.cdf(-d2)
    
    return {'delta': round(delta), 'gamma': round(gamma), 'vega': round(vega), 'theta': round(theta), 'rho': np.round(rho, 4)}

# Problem 1

In [7]:
S, sigma, T, K, r, q = 49, 0.27, 7/12, 52, 0.045, 0.02 

## Monte Carlo

In [8]:
@jit(nopython=True)
def mc_blackscholes(N, T, S, sigma, q, r):
    """
    Monte Carlo simulation of Geometric Brownian motion of underlying in the risk neutral measure
    Only works for path independant options: only computing final state for paths
    Build on numba jit to accelerate performance
    Using antithetic variables to reduce variance
    """
    drift = (r - q - 0.5 * sigma ** 2) * T
    diffusion = sigma * np.sqrt(T)
    halfZ = np.random.standard_normal(N // 2)
    Z = np.concatenate((halfZ, -halfZ))
    return S * np.exp(drift + diffusion * Z)

In [9]:
def mc_bs_european(N, T, S, sigma, q, r, K, opt_type):
    """
    Approximating expected value of option in risk neutral world (discounted mean)
    """
    ST = mc_blackscholes(N, T, S, sigma, q, r)
    disc = np.exp(-r * T)
    V = np.average(np.maximum(ST - K, 0)) if opt_type =="call" else np.average(np.maximum(K - ST, 0))
    return disc * V

## Pricing options

In [10]:
def pricer(pricing_fcn, N, T, S, sigma, q, r, K, option, dS, dsigma, dt, dr, title="option"):
    """
    Pricing option and computing greeks for any pricing method and any shocks 
    """
    # computing price
    V = pricing_fcn(N, T, S, sigma, q, r, K, option)
    # computing prices with shocks 
    V_dS_u = pricing_fcn(N, T, S + dS, sigma, q, r, K, option)
    V_dS_d = pricing_fcn(N, T, S - dS, sigma, q, r, K, option)
    V_dsigma_u = pricing_fcn(N, T, S, sigma + dsigma, q, r, K, option)
    V_dt_d = pricing_fcn(N, T - dt, S, sigma, q, r, K, option)
    V_dr_u = pricing_fcn(N, T, S, sigma, q, r + dr, K, option)

    # computing greeks with shocked prices
    delta = round((V_dS_u - V) / dS)
    gamma = round((V_dS_u - 2 * V + V_dS_d) / (dS ** 2))
    vega = round((V_dsigma_u - V) / dsigma)
    theta = round((V_dt_d - V) / dt)
    rho = round((V_dr_u - V) / dr)

    return pd.DataFrame.from_dict({'price': V, 'delta': delta, 'gamma': gamma, 'vega': vega, 'theta': theta, 'rho': rho}, orient='index', columns=[title])


In [11]:
N = 10_000_000                              # number of paths
dS, dsigma, dt, dr = 1, 0.01, 0.01, 0.01    # shock values

In [12]:
euro_call = pricer(mc_bs_european, N, T, S, sigma, q, r, K, 'call', dS, dsigma, dt, dr, title="call")
euro_call

,call
price,3.052988
delta,0.467500
gamma,0.038700
vega,14.639200
theta,-3.962300
rho,11.246100


In [13]:
euro_put = pricer(mc_bs_european, N, T, S, sigma, q, r, K, 'put', dS, dsigma, dt, dr, title="put")
euro_put

,put
price,5.274805
delta,-0.520000
gamma,0.039600
vega,14.552700
theta,-2.661200
rho,-18.230700


## Comparing with closed form solutions

In [14]:
euro_call_analytical = pd.DataFrame.from_dict(
    {**{'price': blackscholes_analytical(S, K, T, sigma, r, q, "call")}, **blackscholes_greeks(S, K, T, sigma, r, q, "call")}, 
    orient='index', 
    columns=["call_analytical"]
)

In [15]:
pd.concat([euro_call, euro_call_analytical], axis=1)

,call,call_analytical
price,3.052988,3.0534
delta,0.467500,0.4492
gamma,0.038700,0.0388
vega,14.639200,14.6608
theta,-3.962300,-2.9801
rho,11.246100,11.0589


In [16]:
euro_put_analytical = pd.DataFrame.from_dict(
    {**{'price': blackscholes_analytical(S, K, T, sigma, r, q, "put")}, **blackscholes_greeks(S, K, T, sigma, r, q, "put")}, 
    orient='index', 
    columns=["put_analytical"]
)

In [17]:
pd.concat([euro_put, euro_put_analytical], axis=1)

,put,put_analytical
price,5.274805,5.2745
delta,-0.520000,-0.5392
gamma,0.039600,0.0388
vega,14.552700,14.6608
theta,-2.661200,-4.2908
rho,-18.230700,-18.4886


# Problem 2

In [18]:
S, T, K, r, q = 62, 10/12, 66, 0.045, 0.015
P_market = 6.36

In [19]:
N = 10_000_000
# fixing all parameters of mc pricer except sigma (or IV)
price_IV = partial(mc_bs_european, N=N, T=T, S=S, q=q, r=r, K=K, opt_type="put")
# computing error
IV_error = lambda sigma: price_IV(sigma=sigma) - P_market

In [20]:
# secant method with tol= 1e-6
result = root_scalar(IV_error, x0=0.2, x1=0.3, bracket=[0.01, 1], method='secant', xtol=1e-6)
IV = result.root
f"{round(IV):.2%}"

'22.51%'

In [21]:
# making sure that result is close to price analytically
blackscholes_analytical(S, K, T, IV, r, q, opt_type='put')

6.3608

# Problem 3

In [22]:
def trinomial_tree_american(N, T, S, sigma, q, r, K, opt_type):
    """
    Trinomial Tree pricer for vanilla american options 
    """
    # time jumps and price jumps
    dt = T / N
    u = np.exp(sigma * np.sqrt(3 * dt))
    d = 1 / u
    m = 1

    # risk neutral probabilities
    pu = 1/6 + (r - q - sigma ** 2 / 2) * np.sqrt(dt / (12 * sigma ** 2))
    pm = 2/3
    pd = 1/6 - (r - q - sigma ** 2 / 2) * np.sqrt(dt / (12 * sigma ** 2))

    # discount rates
    disc_r = np.exp(-r * dt)
    
    # vanilla payoff
    payoff = lambda S: np.maximum(0, S - K) if opt_type == "call" else np.maximum(0, K - S)

    # states of underlying at maturity
    St = S * u ** np.arange(-N, N+1, 1)
    # pricing option at maturity for each state
    v = payoff(St)

    # backwardation
    for i in range(N, 0, -1):
        # calculating expected value if continuation (no exercise)
        cont = pu * v[2:] + pm * v[1:-1] + pd * v[:-2]
        # getting states of values at next step (removing first and last value)
        St = St[1:-1]
        # getting exercise payoff
        exer = payoff(St)
        # getting next state value
        v = disc_r * np.maximum(cont, exer)
        
    return v[0]

In [23]:
S, sigma, T, K, r, q = 33, 0.24, 10/12, 32, 0.045, 0.02

In [24]:
Ns = [40, 80, 160, 320, 640, 1280]

In [25]:
calls = pd.concat(
    [pricer(trinomial_tree_american, N, T, S, sigma, q, r, K, 'call', dS, dsigma, dt, dr, title=f"call-{N}") for N in Ns], 
    axis=1
)
calls

,call-40,call-80,call-160,call-320,call-640,call-1280
price,3.660196,3.653535,3.648283,3.652856,3.651106,3.651604
delta,0.625600,0.651500,0.649300,0.647800,0.649300,0.649700
gamma,0.002000,0.043000,0.050400,0.046400,0.050700,0.051200
vega,11.179300,11.215700,11.104800,11.137800,11.130200,11.170700
theta,-1.965300,-1.970900,-1.943600,-1.959600,-1.952800,-1.964400
rho,14.250500,14.261500,14.267900,14.261800,14.264300,14.263700


In [26]:
puts = pd.concat(
    [pricer(trinomial_tree_american, N, T, S, sigma, q, r, K, 'put', dS, dsigma, dt, dr, title=f"call-{N}") for N in Ns], 
    axis=1
)
puts

,call-40,call-80,call-160,call-320,call-640,call-1280
price,2.078403,2.074752,2.070933,2.075409,2.074193,2.074768
delta,-0.366700,-0.344700,-0.346400,-0.347700,-0.346500,-0.346200
gamma,0.012300,0.047300,0.054100,0.050700,0.054200,0.054500
vega,11.246600,11.287300,11.212800,11.224700,11.223700,11.254700
theta,-1.310500,-1.319300,-1.298600,-1.311700,-1.307300,-1.316800
rho,-9.227300,-9.103200,-9.042900,-9.059500,-9.041600,-9.040800


# Problem 4

In [27]:
S, T, K, r, q = 56, 8/12, 60, 0.045, 0.015
P_market = 6.95

## i)

In [28]:
IV_guess = 0.25
N_start = 10
tol = 1e-6

In [29]:
P_last = 0
P_curr = trinomial_tree_american(N_start, T, S, IV_guess, q, r, K, 'put')
N = N_start
while abs(P_curr - P_last) > tol:
    N *= 2
    P_last = P_curr
    P_curr = trinomial_tree_american(N, T, S, IV_guess, q, r, K, 'put')
N_fixed = N
N_fixed

320

In [30]:
trinomial_tree_american(N_fixed, T, S, IV_guess, q, r, K, 'put')

6.350841130517868

## ii)

In [31]:
# forward price
F = S * np.exp((r - q) * T)

In [32]:
def SD_approx(P_market, F, T, r, K):
    """
    Implied volatility approximation by Stefanica and Radoicic
    """
    
    def gamma_fcn(y, R):
        A = (np.exp((1 - 2 / np.pi) * y) - np.exp(-(1 - 2 / np.pi) * y)) ** 2
        B = 4 * (np.exp((2 / np.pi) * y) + np.exp(-(2 / np.pi) * y)) - 2 * np.exp(-y) * (np.exp((1 - 2 / np.pi) * y) + np.exp(-(1 - 2 / np.pi) * y)) * (np.exp(2 * y) + 1 - R ** 2)
        C = np.exp(-2 * y) * (R ** 2 - (np.exp(y) - 1) ** 2) * ((np.exp(y) + 1) ** 2 - R ** 2)
        return  -np.pi / 2 * np.log(2 * C / (B + np.sqrt(B ** 2 + 4 * A * C)))
    
    def A(y):
        return  1 / 2 * (1 + np.sign(y) * np.sqrt(1 - np.exp(-(2 / np.pi) * y ** 2)))
        
    y = np.log(F / K)
    alpha_P = P_market / (K * np.exp(-r * T))
    R = 2 * alpha_P + np.exp(y) - 1
    gamma = gamma_fcn(y, R)

    if y >= 0:
        P_0 = K * np.exp(-r * T) * (1 / 2 - np.exp(y) * A(-np.sqrt(2 * y)))
        
        if P_market <= P_0:
            return (np.sqrt(gamma + y) - np.sqrt(gamma - y)) / np.sqrt(T)
            
        else:
            return (np.sqrt(gamma + y) + np.sqrt(gamma - y)) / np.sqrt(T)

    else:
        P_0 = K * np.exp(-r * T) * (A(np.sqrt(-2 * y)) - np.exp(y) / 2) 
        if P_market <= P_0:
            return (-np.sqrt(gamma + y) + np.sqrt(gamma - y)) / np.sqrt(T)
        else:
            return (np.sqrt(gamma + y) + np.sqrt(gamma - y)) / np.sqrt(T)       

In [33]:
# approximated IV by SD approach
IV_SD = SD_approx(P_market, F, T, r, K)
IV_SD

0.29458765639080964

In [34]:
N_fixed
# fixing all parameters of mc pricer except sigma (or IV)
price_IV = partial(trinomial_tree_american, N=N_fixed, T=T, S=S, q=q, r=r, K=K, opt_type="put")
# computing error and storing results
secant_vals = []
def IV_error_store(sigma):
    P_IV = price_IV(sigma=sigma)
    secant_vals.append((sigma, P_IV))
    return P_IV - P_market

In [35]:
# two initial values (IV SD pm 2%)
x0 = IV_SD - 0.02
x1 = IV_SD + 0.02
# secant method with tol= 1e-4
result = root_scalar(IV_error_store, x0=x0, x1=x1, bracket=[0.01, 1], method='secant', xtol=1e-4)
IV = result.root
f"{round(IV):.2%}"

'28.43%'

In [36]:
# iteration results
pd.DataFrame(secant_vals, columns=["IV", "Price"]).T

,0,1,2,3
IV,0.274588,0.314588,0.284147,0.284277
Price,6.780491,7.489786,6.947679,6.949958
